In [2]:
import numpy as np
import string
from tensorflow.keras.models import load_model
# Cargar el modelo desde el archivo
model = load_model('modelo_entrenado.h5')

#En sentence va el dminio semilla
generated = ""
sentence="www.google.com"
maxlen=45



alfabeto = string.ascii_lowercase + "0123456789-._ "
print(alfabeto)

chars = sorted(list(set(alfabeto)))
print("Total chars:", len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))



def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



if len(sentence) < 45:
    sentence = sentence.ljust(45)

    
    
    
for diversity in [ 0.5]:
        print("...Diversity:", diversity)

        

print("...Generated: ", generated)
print("...Diversity:", diversity)
#Si varias el 180000 esa cantidad entre 45 y esa va a ser la cantidad de dminios que nereraras
for i in range(180000):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char

print("...Generated: ", generated)
print()

abcdefghijklmnopqrstuvwxyz0123456789-._ 
Total chars: 40
...Diversity: 0.5
...Generated:  
...Diversity: 0.5


<ipython-input-2-ed36784de910>:28: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  caotaip..oe. n                                 a la.ko                                        abeie.eo                                       edorrmc..ov                                  aaooee-..oe. r  .                            5 lcca.ro                                      e rn.ao                                        aspetc.oog b  y  c  c  c  c  c  c  .                                          esorrcclc.iu                                 o taauaaaa.io                                   a i..yp                                        bboe.crm                                     a ao.ko                                        bsnrrcc.ro                                    aoigoiui..oo. u  .                            b otpdne..oe                                   capeoh.eo                                      bccpncao..oog                                   aaeoogtcmoe..uo                                baciaiaaoo..oog                                   c1aaic..oo         

In [3]:
import pandas as pd


# Longitud de cada segmento
longitud_segmento = 45

# Calcular el número de segmentos
num_segmentos = len(generated) // longitud_segmento

# Crear una lista para almacenar los segmentos
segmentos = [generated[i:i + longitud_segmento] for i in range(0, len(generated), longitud_segmento)]

# Crear el DataFrame "Prueba"
df = pd.DataFrame({"Segmento": segmentos})

# Imprimir el DataFrame
print(df)

                                           Segmento
0     caotaip..oe. n                               
1       a la.ko                                    
2         abeie.eo                                 
3           edorrmc..ov                            
4           aaooee-..oe. r  .                      
...                                             ...
3995  btcuu..oo. r  .                              
3996      srmcirite..oo                            
3997      oaalaieai..oo                            
3998      c ar.eo                                  
3999        bsderclcg.com                          

[4000 rows x 1 columns]


In [4]:
import pandas as pd

def arreglar_dominio(segmento):
    # Eliminar espacios al principio
    segmento_arreglado = segmento.strip()

    # Encontrar el primer espacio después de encontrar letras y cortar ahí
    for i, char in enumerate(segmento_arreglado):
        if char.isspace() and any(c.isalpha() for c in segmento_arreglado[:i]):
            segmento_arreglado = segmento_arreglado[:i]
            break

    return segmento_arreglado



# Unir todos los caracteres de cada fila sin espacios
df["Segmento_unido"] = df["Segmento"].str.replace(" ", "")

# Aplicar la función arreglar_dominio a la columna "Segmento_unido"
df["Segmento_arreglado"] = df["Segmento_unido"].apply(arreglar_dominio)

# Eliminar la columna "Segmento_unido" si ya no es necesaria
df.drop(columns=["Segmento_unido"], inplace=True)

# Imprimir el DataFrame resultante
print(df)


                                           Segmento Segmento_arreglado
0     caotaip..oe. n                                     caotaip..oe.n
1       a la.ko                                                 ala.ko
2         abeie.eo                                            abeie.eo
3           edorrmc..ov                                    edorrmc..ov
4           aaooee-..oe. r  .                           aaooee-..oe.r.
...                                             ...                ...
3995  btcuu..oo. r  .                                     btcuu..oo.r.
3996      srmcirite..oo                                  srmcirite..oo
3997      oaalaieai..oo                                  oaalaieai..oo
3998      c ar.eo                                               car.eo
3999        bsderclcg.com                                bsderclcg.com

[4000 rows x 2 columns]


In [7]:
import pandas as pd

# Crear una nueva columna llamada "class" con valores de 1
df['class'] = 1

# Seleccionar solo la columna "Segmento_arreglado" y la nueva columna "class"
data = df[['Segmento_arreglado', 'class']]

# Renombrar la columna "Segmento_arreglado" como "domain"
data = data.rename(columns={'Segmento_arreglado': 'domain'})

# Guardar los datos en un archivo CSV llamado "archivo.csv"
data.to_csv('4kdomain.csv', index=False)
